# Event finder & raw data checker

This notebook will find competitive cellular "events" in the simplest definition (i.e. a loser cell apoptosis) and return information about the spatiotemporal distrubition of counterpart competitive events (i.e. winner cell mitosis)

Contents:

- Load modules
- Load cell finding functions
- Set experiment data path
- Load image data
- Load tracking data
- Isolate one track of interest (target track)
- Find corresponding tracks/events of interest within a given spatiotemporal range (E.g. if target track is Scr apoptosis, then find all nearby wild-type mitosis)
- Apply necessary coordinate shift for viewer
- Set points and regions of interest for highlighting in Napari viewer
- Launch Napari image viewer if desired

To-do:

- [ ] Reorganise structure so that scan over all R then filter
- [ ] Create radial bins and sort tracks into them
- [ ] Create temporal bins
- [ ] Figure out how to plot said bins into heat map image
- [ ] Check redesigned script against raw data 

In [1]:
import napari
import btrack
import numpy as np
from skimage.io import imread
import os
print("Napari version no.:", napari.__version__)
print("btrack version no.:", btrack.__version__)
from btrack.utils import import_HDF, import_JSON, tracks_to_napari
from tqdm.notebook import tnrange, tqdm
import matplotlib.pyplot as plt

Napari version no.: 0.4.0
btrack version no.: 0.4.0


### Functions to measure local neighbourhood for cells of interest

In [27]:
def euclidean_distance(target_track, other_track, frame):
    try:
        idx0 = target_track.t.index(find_apoptosis_time(target_track, index = False)) ## could also do just ## apop_index
        idx1 = other_track.t.index(frame)
    except:
        return np.inf
    
    dx = target_track.x[idx0] - other_track.x[idx1]
    dy = target_track.y[idx0] - other_track.y[idx1]
    
    return np.sqrt(dx**2 + dy**2)

def find_apoptosis_time(target_track, index): ### if index is set to True then the index of the apoptotic time (wrt target_track) is returned
    for i, j in enumerate(target_track.label):
        if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
            apop_index = i
            break
    apop_time = target_track.t[apop_index]
    if index == True: 
        return apop_index
    else: 
        return apop_time

def find_nearby_wt_mitosis(target_track, delta_t, radius):
    frame = find_apoptosis_time(target_track, index = False) + delta_t 
    dividing_states = ('PROMETAPHASE', 'METAPHASE', 'DIVIDE')
    wt_tracks_in_radius = [wt_track for wt_track in wt_tracks if wt_track.in_frame(frame) if euclidean_distance(target_track, wt_track, frame)<radius]
    wt_mitosis_in_radius = [wt_track for wt_track in wt_tracks if wt_track.in_frame(frame) if euclidean_distance(target_track, wt_track, frame)<radius if wt_track.label[wt_track.t.index(frame)] in dividing_states]
  
    return wt_tracks_in_radius, wt_mitosis_in_radius


def maximum_xyt(i):
    #### BEWARE xy coord switch!     
    if i == 'x':
        max_x = int((max([max(i.y) for i in wt_tracks])))
        return max_x
    if i == 'y':
        max_y = int((max([max(i.x) for i in wt_tracks])))
        return max_y
    if i == 't':
        max_t = int((max([max(i.t) for i in wt_tracks])))
        return max_t

### Set experiment data path 

In [3]:
# print("Input experiment number")
# experiment_no = input()
# root_path = os.path.join('/home/nathan/data/kraken/h2b/giulia/', experiment_no)
root_path = '/home/nathan/data/kraken/h2b/giulia/GV0807'  ## this overwrites input option for ease 
gfp_path = os.path.join(root_path, 'Pos3/stacks/gfp.tif')
rfp_path = os.path.join(root_path, 'Pos3/stacks/rfp.tif')
bf_path = os.path.join(root_path, 'Pos3/stacks/bf.tif')
tracks_path = os.path.join(root_path, 'Pos3/Pos3_aligned/HDF/segmented.hdf5')

### Load image data

In [4]:
gfp = imread(gfp_path)

In [5]:
rfp = imread(rfp_path)

In [ ]:
bf = imread(bf_path)

### Load tracking data

In [6]:
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks
print("Tracks loaded")

[INFO][2020/12/07 05:43:29 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2020/12/07 05:43:29 PM] Loading tracks/obj_type_1
[INFO][2020/12/07 05:43:31 PM] Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
[INFO][2020/12/07 05:43:34 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2020/12/07 05:43:34 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2020/12/07 05:43:34 PM] Loading tracks/obj_type_2
[INFO][2020/12/07 05:43:35 PM] Loading objects/obj_type_2 (12115, 5) (8894 filtered: area>=100)
[INFO][2020/12/07 05:43:35 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5


Tracks loaded


### Isolate one track of interest

In [7]:
## isolate one target scribble track of interest
print("input scribble track of interest ID")
cell_ID = int(input())
index = [i for i, scr_tracks in enumerate(scr_tracks) if scr_tracks.ID == cell_ID][0]
target_track = scr_tracks[index]##### Show the first classification of each track
apop_index = find_apoptosis_time(target_track, index = True)

input scribble track of interest ID
17


# Find cells of interest

In [8]:
radius = 1000
delta_t = -100

In [28]:
wt_tracks_in_radius, wt_mitosis_in_radius = find_nearby_wt_mitosis(target_track, delta_t, radius)

display ID for all tracks found in radius 

In [30]:
print("ID for all tracks in radius:", [j.ID for i, j in enumerate(wt_tracks_in_radius)])
print("ID for all mitoses in radius:", [j.ID for i, j in enumerate(wt_mitosis_in_radius)])

ID for all tracks in radius: [29, 33, 42, 4, 94, 109, 132, 143, 148, 154, 157, 161, 169, 174, 175, 183, 185, 189, 218, 267, 295, 310, 311, 312, 316, 319, 320, 330, 331, 333, 335, 336, 340, 339, 341, 346, 345, 343, 342, 344, 347, 348, 352, 353, 357, 358, 367, 368, 369, 371, 372, 377, 378, 380, 381, 383, 384, 389, 393, 397, 398, 399, 400, 403, 404, 405, 407, 411, 412, 413, 414, 418, 419, 420, 421, 428, 430, 441, 440, 437, 436, 445, 446, 448, 450, 453, 454, 455, 456, 461, 462, 476, 472, 471, 468, 469, 478, 483, 485, 486, 490, 493, 495, 494, 492, 491, 496, 497, 501, 503, 513, 512, 509, 508, 507, 518, 519, 526, 527, 528, 529, 533, 536, 535, 534, 539, 540, 541, 542, 543, 545, 550, 549, 554, 555, 562, 561, 560, 566, 563, 568, 575, 584, 585, 587, 588, 589, 590, 591, 593, 594, 595, 600, 601, 602, 603, 604, 605, 606, 609, 610, 612, 613, 615, 618, 621, 622, 628, 629, 631, 632, 634, 635, 640, 642, 644, 647, 648, 649, 650, 651, 652, 657, 658, 664, 666, 670, 673, 674, 675, 676, 678, 682, 683, 686, 6

In [32]:
wt_mitosis_in_radius[1]

,ID,t,x,y,z,parent,root,state,generation,dummy
0,333,446,16.677687,553.859497,0.0,333,333,3,0,False
1,333,447,23.115740,556.861084,0.0,333,333,3,0,False
2,333,448,22.474403,555.300354,0.0,333,333,3,0,False
3,333,449,22.949110,551.796448,0.0,333,333,3,0,False
4,333,450,24.454988,547.085144,0.0,333,333,3,0,False
5,333,451,23.155661,540.688660,0.0,333,333,3,0,False
6,333,452,22.422052,536.868835,0.0,333,333,3,0,False
7,333,453,25.693014,532.034912,0.0,333,333,0,0,False
8,333,454,24.357143,529.398193,0.0,333,333,1,0,False
9,333,455,22.791594,525.802124,0.0,333,333,0,0,False


### Necessary coordinate shift for raw data viewing (due to stablised images used on tracks)

In [11]:
### finding coord range of aligned images, coords switched already
align_x_range, align_y_range = gfp.shape[2], gfp.shape[1]
### finding maximum extent of tracking coords
tracks_x_range = round(max([max(track.x) for track in wt_tracks]))
tracks_y_range = round(max([max(track.y) for track in wt_tracks])) + 2 ## sort this lazy hack out later

### coord switch
tmp = tracks_y_range
tracks_y_range = tracks_x_range
tracks_x_range = tmp

print("tracks range:", (tracks_x_range), (tracks_y_range))
print("aligned image range:", (align_x_range), (align_y_range))

shift_x = int((align_x_range - tracks_x_range)/2)
shift_y = int((align_y_range - tracks_y_range)/2)

print("shift in x and y:", shift_x, shift_y)

wt_data, properties, graph = tracks_to_napari(wt_tracks, ndim = 2)
scr_data, properties, graph = tracks_to_napari(scr_tracks, ndim = 2)

tmp = wt_data[:,2].copy() ## copy the true_y coord
wt_data[:,2] = wt_data[:,3]  ##assign the old_y coord as the true_x
wt_data[:,3] = tmp ## assign the old_x as true_y

wt_data[:,2] += shift_y ## TRUE_Y (vertical axis)
wt_data[:,3] += shift_x ## TRUE_X (horizontal axis)

tmp = scr_data[:,2].copy()
scr_data[:,2] = scr_data[:,3]
scr_data[:,3] = tmp

scr_data[:,2] += shift_y ## TRUE_Y (vertical axis)
scr_data[:,3] += shift_x ## TRUE_X (horizontal axis)

print("coordinate shift applied")

tracks range: 1600 1200
aligned image range: 1739 1377
shift in x and y: 69 88
coordinate shift applied


### Define reference points if needed

In [12]:
### add_points needs to be txy
apop_event = target_track.t[apop_index], target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x ## with transposed shift
print("txy of apoptotic event:", apop_event)

txy of apoptotic event: (730, 638.5511474609375, 1158.223876953125)


adding circle reference shape

In [13]:
## apop_circle is the bounding box of the ellipse
apop_circle = [(apop_event[0]+delta_t, apop_event[1]-radius, apop_event[2]-radius), 
                (apop_event[0]+delta_t, apop_event[1]+radius, apop_event[2]-radius), 
                (apop_event[0]+delta_t, apop_event[1]+radius, apop_event[2]+radius), 
                (apop_event[0]+delta_t, apop_event[1]-radius, apop_event[2]+radius)]


adding corresponding wt_mitoses

In [92]:
def plot_mitoses():
    t_m, x_m, y_m = np.zeros(len(wt_mitosis_in_radius)), np.zeros(len(wt_mitosis_in_radius)), np.zeros(len(wt_mitosis_in_radius))
    mito_events = np.zeros((len(wt_mitosis_in_radius), 3)) ## 3 because of the 3 cartesian coords 
    for i, wt_mitosis in enumerate(wt_mitosis_in_radius): ## this assumes that the wt_mitosis goes through mitosis at its final frame (true of btrack classifcations) BUT this is not the correct, some tracks go thru mitosis not before theirend?
        mito_index = [j for j, k in enumerate(wt_mitosis.t) if k == apop_event[0]+delta_t][0] ### [0] bc first item of list comprehension

        t_m[i], x_m[i], y_m[i] = wt_mitosis.t[mito_index], wt_mitosis.x[mito_index]+shift_y, wt_mitosis.y[mito_index]+shift_x ## plus transposed coordinate shift
        mito_events[i] = t_m[i], x_m[i], y_m[i]
    return viewer.add_points(mito_events, name = "Mitosis events", symbol = "cross")

### Launch napari to check against raw data 

In [93]:
with napari.gui_qt():
    viewer = napari.Viewer()
    
    #viewer.add_image(bf)
    viewer.add_image(gfp, name="gfp", blending = "additive", colormap = "green")
    viewer.add_image(rfp, name="rfp", contrast_limits = (0, 50), blending = "additive", colormap = "red")
    
    viewer.add_tracks(wt_data)
    viewer.add_tracks(scr_data)
    viewer.add_points(apop_event, name = "Apoptosis event")
    viewer.add_shapes(apop_circle, opacity = 0.2, shape_type = 'ellipse', face_color = 'y', name = 'Apoptosis event')
    #viewer.add_points(mito_events, name = "Mitosis events", symbol = "cross")
    plot_mitoses()